# Hello, `vizarr`

This example explains the basic usage of `vizarr` [anywidget](https://github.com/manzt/anywidget).

### Create a multiscale OME-Zarr

The easiest way to use `vizarr` in a jupyter notebook is to have an OME-Zarr compliant image. OME-Zarr is a developing standard from the OME-Community and currently considered experimental as it is under active development. The specification can be found [here](https://github.com/ome/omero-ms-zarr/blob/master/spec.md). 
 
Creating an OME-Zarr for your own images can be accomplished by using Bio-Formats, `bioformats2raw` with the `--file_type=zarr --dimension-order=XYZCT` options and adding `omero` metadata to the root attrs of the resulting multiscale group. 

For convenience, we have included a simple function to create a multiscale OME-Zarr below.

In [ ]:
from create_fixture import create_ome_zarr
create_ome_zarr("astronaut.zarr") # creates an example OME-Zarr in the current directory

### Open OME-Zarr in jupyter with `zarr-python`

The root of an OME-Zarr is a group that aheres to the `multiscales` zarr extension (if pyramidal) and also contains `omero` metadata describing _how_ to render the image. Here we open the OME-Zarr as a `zarr.Group` using `zarr-python`. 

In [ ]:
import zarr
multiscale_astronaut = zarr.open("astronaut.zarr", mode="r") # open the zarr created above in jupyter kernel

### View the image in `vizarr`

The vizarr Python API allows basic viewing of large zarr datasets in computational notebooks. The `vizarr.Viewer` allows adding images to the view with the `.add_image` API. The `source` field must be a `zarr.Array` or `zarr.Group`. If the `zarr.Group` is for an OME-Zarr, `source` is the only required field. If it is an `zarr.Array` or `zarr.Group` that is not OME-Zarr, additional fields are required for rendering (described below).

In [ ]:
import vizarr

viewer = vizarr.Viewer()
viewer.add_image(source=multiscale_astronaut, name="astronaut")
viewer

### Create a generic array layer

`vizarr` also supports custom `zarr.Array` and `zarr.Group`s, but requires additional metadata to render. Viewing a custom `zarr` has the following requirements:

- The last two dimensions of the `zarr.Array` must be `YX`.
- If an `zarr.Array` has more than 2D dimensions, non-YX axis much have a `chunksize` of `1`.
- If a `zarr.Group` is provided, it must implement the `multiscales` specification and subresolution arrays must adhere to the same `zarr.Array` requirements outlined above.

In [ ]:
import numpy as np

# Create in-memory numpy array
arr = np.random.randint(0, 255, (1024, 1024), dtype=np.uint8)

viewer.add_image(
    source=zarr.array(arr), # make sure it's a zarr
    opacity=0.5,   # optional, default 1
    name="noise",  # optional, default image_x
)

# view canvas above

### Use `channel_axis` to choose axis to additively blend

Vizarr was designed to support viewing a subset of multiscale OME-Zarr images, but it can support blending 2D slices of generic `zarr.Array`s. For a `zarr.Array` with more than 2 dimensions, a `channel_axis` must be specified to tell vizarr which dimension to split into separate blended channels. 

Here we will load one of the sub-resolution `zarr.Array`s from the OME-Zarr we created and view it with Vizarr.

In [ ]:
z_arr = zarr.open('astronaut.zarr').get('2')
z_arr.shape

The channel axis for an OME-Zarr is always `1`, but loading the image with a different channel axis will load a single plane.

In [ ]:
viewer = vizarr.Viewer()
viewer.add_image(source=z_arr, channel_axis=0)
viewer

You can specify other rendering properties to tell vizarr how to render the channels along the specified axis.

In [ ]:
viewer = vizarr.Viewer()
viewer.add_image(
    source=z_arr,
    name="CYM astronaut", # Image name
    channel_axis=1, # size == 3, lists below must be of same length if provided
    colors=["#FFFF00", "#FF00FF", "#00FFFF"],   # optional
    names=["yellow", "magenta", "cyan"],        # optional, default channel_X
    visibilities=[True, False, True],           # optional, default True
    contrast_limits=[[0, 1] for _ in range(3)], # optional, but recommended
)
viewer